In [1]:
import pandas as pd 
import numpy as np
import string 
import re
import swifter
import matplotlib.pyplot as plt
dataset = pd.read_csv('data_final.csv', encoding='utf-8')
dataset.head()

,Unnamed: 0,date,username,tweet,nlikes,label
0,0,4/30/2022 5:22,airisk_,@tvindonesiawkwk Kalo cewe jual video syur cep...,10.0,not urgent
1,1,4/30/2022 4:15,idntimes,A (23) adalah penyintas korban percobaan keker...,0.0,urgent
2,2,4/30/2022 4:10,tudungsaaji,@scrxnb UU TPKS udah bisa digunakan buat menje...,0.0,not urgent
3,3,4/30/2022 1:03,eLAmaravati,Gak heran banyak anak yang diam ketika mengala...,18.0,not urgent
4,4,4/30/2022 6:55,jawuwawy,"@andreasharsono Sama kaya di kampus saya, korb...",2.0,not urgent


In [2]:
#data_cleansing
#menghapus kolom selain tweet dan label
dataset.drop(
    ['Unnamed: 0','date','username', 'nlikes'], 
    axis=1, 
    inplace=True)
dataset.head()

,tweet,label
0,@tvindonesiawkwk Kalo cewe jual video syur cep...,not urgent
1,A (23) adalah penyintas korban percobaan keker...,urgent
2,@scrxnb UU TPKS udah bisa digunakan buat menje...,not urgent
3,Gak heran banyak anak yang diam ketika mengala...,not urgent
4,"@andreasharsono Sama kaya di kampus saya, korb...",not urgent


In [3]:
#case_folding
dataset['tweet'] = (dataset['tweet'].str.lower())
print(dataset['tweet'])

0      @tvindonesiawkwk kalo cewe jual video syur cep...
1      a (23) adalah penyintas korban percobaan keker...
2      @scrxnb uu tpks udah bisa digunakan buat menje...
3      gak heran banyak anak yang diam ketika mengala...
4      @andreasharsono sama kaya di kampus saya, korb...
                             ...                        
114    @franciscopamz gw masih bingung sih menanggapi...
115    vonis hukuman mati terhadap hendi alias abah h...
116    lagi-lagi kasus kekerasan seksual pada anak te...
117    setan. gue beneran benci bgt sama pelaku keker...
118    @virgitaaa_ @shabrinaf_ silahkan di cek tingka...
Name: tweet, Length: 119, dtype: object


In [4]:
#mendeteksi nilai yang hilang 
dataset.isnull()
#meringkas nilai yang hilang
dataset.isnull().sum().sum()

0

In [5]:
#NLTK 
#import nltk
#nltk.download('punkt')

In [6]:
#tokenizing
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

#menghapus karakter khusus (spesial)
def remove_special_word(word):
    #menghapus tab, baris baru, dan back slice
    word = word.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    #menghapus karakter non ASCII (emoticon, dll)
    word = word.encode('ascii', 'replace').decode('ascii')
    #menghapus mention, link, dan hashtag
    word = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", word).split())
    #menghapus tautan URL yang tidak lengkap
    word = word.replace("http://", " ").replace("https://", " ")
    #menghapus angka
    word = re.sub(r"\d+", "", word)
    #menghapus multispasi
    word = re.sub('\s+',' ',word)
    #menghapus single char
    word = re.sub(r"\b[a-zA-Z]\b", "", word)
    #menghapus tanda baca (pungtuasi)
    word = word.translate(str.maketrans("","",string.punctuation))
    #menghapus spasi di awal dan akhir
    word = word.strip()
    #NLTK tokenize
    word = word_tokenize(word)
    return word 

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org

In [7]:
dataset['tweet_tokens_result'] = dataset['tweet'].apply(remove_special_word)

In [8]:
#NLTK
#import nltk
#nltk.download('stopwords')

In [9]:
#stopword
from nltk.corpus import stopwords
#NLTK stopword
stopwords_list = stopwords.words('indonesian')
#stopword yang ditambahkan secara manual
stopwords_list.extend(['kalo','ya','kok','dan','para','tapi','cepet','adalah','di','yang','dari','merupakan',
                       'ini','lalu','pada','berikut','april','uu','tpks','gasi','cuma','ada','itu','mungkin',
                       'sender','gak','aja','salty','sih','mana','reply','nya','byk','yg','udh','ke','juga',
                       'bentukbentuk','agar','kita','tidak','tahu','ketika','rekan','commuters','sedang',
                       'petugas','mulawarman','dosen','mjb','kak','btw','dia','minyak','angin','udah',
                       'belakang','kakakku','seketika','demi','allah','bgt','ko','kek','anjir','norak',
                       'lu','bgt','lagi','tae','gaperna','emang','kayaknya','ada','pasti','dulu','sampe',
                       'kaya','banget','apalagi','buat','ga','amp','asprinya','hah','anjg','gada','yg', 
                       'komen','portal','hotman','paris','untuk','anggota','dprd','gini','tahun', 
                       'adik','barbie','hsu','artis','senior','dee','hsu','pembawa','acara','sang', 
                       'host','apa','mantan','asisten','iqlima','kim','pribadi','gue','rektorat', 
                       'unmul','muslim','kaos','wali','ayahnya','dosen','nggilani','haloo','selamat', 
                       'spill','lo','pas','absurd','au','you','si','krn','emg','gamau','mlh','blg', 
                       'thank','you','lgsg','block','anjr','artis','doctor','strange','zara','phythian', 
                       'marke','seolaholah','bbc','pun','siapa','ruppks','sahkan','skali','yg','tdk', 
                       'media','klu','wakil','negara','nyampe','musti','ngapak','andil','via','tetapi', 
                       'outta','me','knp','putus','ye','tbtb','nah','njjr','smg','jam','anjinggggggg', 
                       'ojol','trus','username','ig','an','lhoo','nder','her','istighfar','nggak','ngga', 
                       'dr','bang','hai','berkata','littleluv','kesia','pen','bunder','skrg','skors', 
                       'katingnya','tp','kl','ga','kakanya','sujud','kating','adik','fucked','lpm', 
                       'ukm','sekret','td','mas','ntah','hmmm','au','daddy','baby','pegawainya', 
                       'wkwkw','nih','nany','tauu','kitadia','wtf','unfaedah','skli','author','this', 
                       'is','mh','tntng','kampre','tonic','ah','immobility','alhamdulillahnya','semester',
                       'hadu','bener','nganggep','abis','thinking','dah','kyk','skorsing','sosi','silaturahmi', 
                       'klo','damn','im','sorry','ati','relate','dis','krlgue','security','yauda','ta','cli', 
                       'dil','org','deh','wkwk','tamatin','kuliah','dek','as','women','pengacara','wp', 
                       'whatt','yany','gmna','skrng','bnr','ia','dpt','kitadia','lulus','or','skg','lgi', 
                       'gtu','mendptk','guru','ngaji','barbie','kumalasari','curhat','pengacara','kubar', 
                       'ipar','via','kaltim','polsek','bibi','vera','jin','ceri','qorin','popo','oprasi', 
                       'drpd','lbh','tp','yagn','bestie','maparin','era','kirana','dgn','but','what', 
                       'said','oot','tweet','dr','ttg','right','untamesti','gercep','keturunanpa','sekarangtp', 
                       'begitu','tsamara','laporga','krn','santri','hello','mba','sy','jgn','we','have','the', 
                       'ability','to','protect','ours','tks','dishub','jd','ntah','lupi','amber','heard','tmsk', 
                       'man','hustling','pesepakbola','bola','ogah','adam','suspek','lilis','dyah','tokoku','tak', 
                       'chapter','sanji','bintangnya','karakter','favorit','udh','ortu','ad','dri','blg','gk', 
                       'ny','hendi','alias','abah','heni','sukabumi','jawa','barat','jabar','setan','sumpah', 
                       'point','femini','gw','lagilagi','ceweanak','kekerasankekerasan','guys','yaa','at','least',
                       'sm','caihua','karna','tuh','blbalbala','tu','bangett','rep','hadumemang','kayu','rektoran','kondang',
                       'jg','loh','dsbnyaa','gitu','boss','yuk','yahh','mksdnya','ternayta','sbg','ntar','benerbener',
                       'up','tsb','hihi','yah','temanteman','tauu','boss','sepuluh','lakilaki','owalah','krna','galih',
                       'ginanjar','hantu','iparnya','tibatiba','unyuunyu','utk','hehehe','bgtu','kau','msuk','msh','lg',
                       'avraham','grepegrepe','tau','dsb','op','smentara','kluar','monyetmonyet','sebel','ama','std',
                       'dipujapuja','dlu','hayolo','ni','ku','bundir','halhal','diem','mrka','bkn','mgkn'])

#konversi string stopword ke list
stopwords_list.extend(['stopwords'][0].split(' '))

#konversi list stopword ke dictionary
stopwords_list = set(stopwords_list)

In [10]:
#menghapus stopword pada list token
def stopwords_list_remove(text):
    return [term for term in text if term not in stopwords_list]
dataset['tweet_tokens_clean'] = dataset['tweet_tokens_result'].apply(stopwords_list_remove)

In [11]:
print(dataset['tweet_tokens_clean'])

0      [cewe, jual, video, syur, nangkepnya, cowo, pe...
1      [penyintas, korban, percobaan, kekerasan, seks...
2      [menjerat, pelaku, kekerasan, seksual, baca, s...
3      [heran, anak, diam, mengalami, seksual, orang,...
4      [kampus, korban, pelecehan, seksual, sanksi, p...
                             ...                        
114    [bingung, menanggapi, aborsi, risiko, anak, di...
115    [vonis, hukuman, mati, pelaku, pemerkosaan, an...
116                           [kekerasan, seksual, anak]
117    [beneran, benci, pelaku, kekerasan, ngilangin,...
118    [silahkan, cek, tingkat, kekerasan, anak, pere...
Name: tweet_tokens_clean, Length: 119, dtype: object


In [12]:
#pip install sastrawi

In [13]:
#pip install swifter --user

In [14]:
#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
#membuat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
#proses stemming
dictionary = {}

def stem_wrapper(root):
    return stemmer.stem(root)
for word in dataset['tweet_tokens_clean']:
    for root in word:
        if root not in dictionary:
            dictionary[root] = ' '  
print(len(dictionary))
for root in dictionary:
    dictionary[root] = stem_wrapper(root)
    print(root,":" ,dictionary[root])
print(dictionary)

773
cewe : cewe
jual : jual
video : video
syur : syur
nangkepnya : nangkepnya
cowo : cowo
pelaku : laku
kekerasan : keras
seksual : seksual
koruptor : koruptor
prosesnya : proses
penyintas : sintas
korban : korban
percobaan : coba
surabaya : surabaya
berhasil : hasil
melarikan : lari
aksi : aksi
bejat : bejat
temannya : teman
kejadian : jadi
ceritanya : cerita
menjerat : jerat
baca : baca
seingetku : seingetku
kejadiannya : jadi
atur : atur
seenggaknya : enggak
kakaknya : kakak
layanan : layan
pemulihan : pulih
miris : miris
heran : heran
anak : anak
diam : diam
mengalami : alami
orang : orang
tuanya : tua
bilang : bilang
harmonis : harmonis
tinggal : tinggal
dunia : dunia
menganggap : anggap
tua : tua
berhala : berhala
kampus : kampus
pelecehan : leceh
sanksi : sanksi
pelayanan : layan
akademik : akademik
alasannya : alas
dianggap : anggap
bersalah : salah
diajak : ajak
bertemu : temu
malam : malam
lucu : lucu
korbannya : korban
takut : takut
temanku : teman
laki : laki
berani : beran

kamar : kamar
hotel : hotel
diketuk : ketuk
istri : istri
berbincang : bincang
sangka : sangka
kondisi : kondisi
mabuk : mabuk
menyerang : serang
keinget : keinget
parah : parah
ciumin : ciumin
pokoknya : pokok
tendang : tendang
beruntung : untung
lolos : lolos
menyenangkan : senang
menyakitkan : sakit
untung : untung
badanku : badan
gede : gede
jago : jago
kurus : kurus
tipe : tipe
diperkosa : perkosa
sambungnya : sambung
diculik : culik
berangkat : berangkat
kerja : kerja
menodongkan : todong
pisau : pisau
menyuruhnya : suruh
mobil : mobil
keterangan : terang
tersangka : sangka
mengancamnya : ancam
membuka : buka
dalamnya : dalam
asusila : asusila
akibat : akibat
perbuatan : buat
keberatan : berat
melaporkan : lapor
damai : damai
sahabatku : sahabat
namaku : nama
mempercayaimu : percaya
pria : pria
penampilan : tampil
resahkan : resah
lakik : lakik
terlepas : lepas
agama : agama
kagak : kagak
serba : serba
lupa : lupa
sentimental : sentimental
paham : paham
otomatis : otomatis
mreka 

{'cewe': 'cewe', 'jual': 'jual', 'video': 'video', 'syur': 'syur', 'nangkepnya': 'nangkepnya', 'cowo': 'cowo', 'pelaku': 'laku', 'kekerasan': 'keras', 'seksual': 'seksual', 'koruptor': 'koruptor', 'prosesnya': 'proses', 'penyintas': 'sintas', 'korban': 'korban', 'percobaan': 'coba', 'surabaya': 'surabaya', 'berhasil': 'hasil', 'melarikan': 'lari', 'aksi': 'aksi', 'bejat': 'bejat', 'temannya': 'teman', 'kejadian': 'jadi', 'ceritanya': 'cerita', 'menjerat': 'jerat', 'baca': 'baca', 'seingetku': 'seingetku', 'kejadiannya': 'jadi', 'atur': 'atur', 'seenggaknya': 'enggak', 'kakaknya': 'kakak', 'layanan': 'layan', 'pemulihan': 'pulih', 'miris': 'miris', 'heran': 'heran', 'anak': 'anak', 'diam': 'diam', 'mengalami': 'alami', 'orang': 'orang', 'tuanya': 'tua', 'bilang': 'bilang', 'harmonis': 'harmonis', 'tinggal': 'tinggal', 'dunia': 'dunia', 'menganggap': 'anggap', 'tua': 'tua', 'berhala': 'berhala', 'kampus': 'kampus', 'pelecehan': 'leceh', 'sanksi': 'sanksi', 'pelayanan': 'layan', 'akademik

In [15]:
#menerapkan stemming pada dataframe
def get_stemmer(file):
    return [dictionary[root] for root in file]
dataset['tweet_tokens_clean2'] = dataset['tweet_tokens_clean'].swifter.apply(get_stemmer)

print(dataset['tweet_tokens_clean2'])


0      [cewe, jual, video, syur, nangkepnya, cowo, la...
1      [sintas, korban, coba, keras, seksual, surabay...
2      [jerat, laku, keras, seksual, baca, seingetku,...
3      [heran, anak, diam, alami, seksual, orang, tua...
4      [kampus, korban, leceh, seksual, sanksi, layan...
                             ...                        
114    [bingung, tanggap, aborsi, risiko, anak, telan...
115    [vonis, hukum, mati, laku, perkosa, anak, pere...
116                               [keras, seksual, anak]
117    [beneran, benci, laku, keras, ngilangin, nyawa...
118    [silah, cek, tingkat, keras, anak, perempuan, ...
Name: tweet_tokens_clean2, Length: 119, dtype: object


In [16]:
#pip install pandas==1.1.5 --user

In [17]:
#pip install arctic==1.79.2 --user --no-warn-script-location

In [18]:
#conda install -c conda-forge swifter

In [19]:
#mengubah list pada kolom tweet_tokens_clean2 menjadi string ke kolom string_clean
dataset['string_clean'] = dataset['tweet_tokens_clean2'].apply(lambda x: ' '.join(map(str, x)))

In [20]:
dataset

,tweet,label,tweet_tokens_result,tweet_tokens_clean,tweet_tokens_clean2,string_clean
0,@tvindonesiawkwk kalo cewe jual video syur cep...,not urgent,"[kalo, cewe, jual, video, syur, cepet, banget,...","[cewe, jual, video, syur, nangkepnya, cowo, pe...","[cewe, jual, video, syur, nangkepnya, cowo, la...",cewe jual video syur nangkepnya cowo laku kera...
1,a (23) adalah penyintas korban percobaan keker...,urgent,"[adalah, penyintas, korban, percobaan, kekeras...","[penyintas, korban, percobaan, kekerasan, seks...","[sintas, korban, coba, keras, seksual, surabay...",sintas korban coba keras seksual surabaya hasi...
2,@scrxnb uu tpks udah bisa digunakan buat menje...,not urgent,"[uu, tpks, udah, bisa, digunakan, buat, menjer...","[menjerat, pelaku, kekerasan, seksual, baca, s...","[jerat, laku, keras, seksual, baca, seingetku,...",jerat laku keras seksual baca seingetku jadi l...
3,gak heran banyak anak yang diam ketika mengala...,not urgent,"[gak, heran, banyak, anak, yang, diam, ketika,...","[heran, anak, diam, mengalami, seksual, orang,...","[heran, anak, diam, alami, seksual, orang, tua...",heran anak diam alami seksual orang tua bilang...
4,"@andreasharsono sama kaya di kampus saya, korb...",not urgent,"[sama, kaya, di, kampus, saya, korban, peleceh...","[kampus, korban, pelecehan, seksual, sanksi, p...","[kampus, korban, leceh, seksual, sanksi, layan...",kampus korban leceh seksual sanksi layan akade...
...,...,...,...,...,...,...
114,@franciscopamz gw masih bingung sih menanggapi...,not urgent,"[gw, masih, bingung, sih, menanggapi, aborsi, ...","[bingung, menanggapi, aborsi, risiko, anak, di...","[bingung, tanggap, aborsi, risiko, anak, telan...",bingung tanggap aborsi risiko anak telantar al...
115,vonis hukuman mati terhadap hendi alias abah h...,urgent,"[vonis, hukuman, mati, terhadap, hendi, alias,...","[vonis, hukuman, mati, pelaku, pemerkosaan, an...","[vonis, hukum, mati, laku, perkosa, anak, pere...",vonis hukum mati laku perkosa anak perempuan h...
116,lagi-lagi kasus kekerasan seksual pada anak te...,urgent,"[lagilagi, kasus, kekerasan, seksual, pada, an...","[kekerasan, seksual, anak]","[keras, seksual, anak]",keras seksual anak
117,setan. gue beneran benci bgt sama pelaku keker...,urgent,"[setan, gue, beneran, benci, bgt, sama, pelaku...","[beneran, benci, pelaku, kekerasan, ngilangin,...","[beneran, benci, laku, keras, ngilangin, nyawa...",beneran benci laku keras ngilangin nyawa manus...


In [21]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   tweet                119 non-null    object
 1   label                119 non-null    object
 2   tweet_tokens_result  119 non-null    object
 3   tweet_tokens_clean   119 non-null    object
 4   tweet_tokens_clean2  119 non-null    object
 5   string_clean         119 non-null    object
dtypes: object(6)
memory usage: 5.7+ KB


In [22]:
#dataset.to_csv("text_preprocessing1.csv")